In [ ]:
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

In [ ]:
url = "https://pubgeo.zwemwater.nl/geoserver/zwr_public/wfs"
wfs = WebFeatureService(url=url)

In [ ]:
def load_wfs_layer(layer_name):
    params = dict(service='WFS', version="2.0.0", request='GetFeature',
      typeName=layer_name, outputFormat='json', srsName="EPSG:4326")
    wfs_request_url = Request('GET', url, params=params).prepare().url
    return gpd.read_file(wfs_request_url)

## Zwemplekken

In [ ]:
zwemplekken = load_wfs_layer("zwr_public:zwemplekken")
zwemplekken = zwemplekken.drop(columns=['id', 'datum']).drop_duplicates(subset=["zwemwaterlocatie_id"]).set_index('key_id')
zwemplekken

In [ ]:
zwemplekken_details = load_wfs_layer("zwr_public:zwemplekken_details")
zwemplekken_details = zwemplekken_details.drop(columns=["id", "info_id", "geometry", "datum"])
zwemplekken_details

In [ ]:
base_data = zwemplekken_details.copy()
base_data = base_data[["zwemwaterlocatie_id", "naam", "korte_naam", "status", "tekst", "info_filename", "website"]]
base_data

In [ ]:
zwemplekdata = zwemplekken.merge(base_data, left_on="zwemwaterlocatie_id", right_on="zwemwaterlocatie_id", how="left")
zwemplekdata['lat'] = zwemplekdata['geometry'].y
zwemplekdata['lon'] = zwemplekdata['geometry'].x
zwemplekdata = zwemplekdata.drop(columns=["korte_naam_x", "naam_x", "status_x", "geometry"])
zwemplekdata = zwemplekdata.rename(columns={
    "naam_y": "naam",
    "korte_naam_y": "korte_naam",
    "status_y": "status"
})
zwemplekdata.sort_values(by=["naam"], inplace=True)
zwemplekdata

In [ ]:
eu_designations = load_wfs_layer("zwr_public:eustatussen")
eu_designations = eu_designations[eu_designations['jaar'] == 2023].drop(columns=['type_eu_status_id', 'geometry', 'omschrijving', 'id'])
eu_designations

In [ ]:
eu_current_designations = eu_designations.query("jaar == 2023")
eu_current_designations = eu_current_designations.drop(columns=["key_id", "jaar"])
eu_current_designations
data = zwemplekdata.merge(eu_current_designations, left_on="zwemwaterlocatie_id", right_on="zwemwaterlocatie_id", how="left")
data.fillna({'code': 0}, inplace=True)
data

In [ ]:
measurements = load_wfs_layer("zwr_public:resultaatsen")
measurements = measurements.drop(columns=['id', 'datum_geplande_monstername', 'monitoring_datum_id', 'monitoring_plan_id', 'monster_id', 'type_object_id', 'geometry'])
measurements = measurements.sort_values(["zwemwaterlocatie_id", "object_begin_tijd", "type_object_code"])
measurements = measurements.drop_duplicates(subset=["zwemwaterlocatie_id", "type_object_code"], keep="last")
measurements


In [ ]:
measure = measurements.pivot_table(values='numerieke_waarde', index='zwemwaterlocatie_id', columns='type_object_code')
measure

In [ ]:
data = data.merge(measure, on="zwemwaterlocatie_id", how="left")
data

In [ ]:
zwemplek_voorziening = load_wfs_layer("zwr_public:zwemplek_voorziening")
zwemplek_voorziening

In [ ]:
voorzieningen = zwemplek_voorziening.groupby('zwemwaterlocatie_id')['voorziening_type_id'].agg(list).reset_index()
voorzieningen

In [ ]:
data = data.merge(voorzieningen, on="zwemwaterlocatie_id", how="left")
data.rename(columns={
    "naam": "name",
    "korte_naam": "alternate_name",
    "status": "current_status",
    "tekst": "description",
    "info_filename": "photos",
    "code": "eu_designation",
    "E_COLI": "e_coli",
    "INTTNLETRCCN": "int_ent",
    "voorziening_type_id": "voorzieningen"
}, inplace=True)
data.set_index("zwemwaterlocatie_id", inplace=True)
data

# Voorzieningen

In [ ]:
unique_amenities = zwemplek_voorziening.copy()
unique_amenities = unique_amenities.drop_duplicates("code").drop(columns=['id', 'taal', 'zwemplek_id', 'zwemwaterlocatie_id', 'geometry'])
unique_amenities = unique_amenities[['voorziening_type_id', 'omschrijving', 'groep']].set_index('voorziening_type_id')
unique_amenities

## EU Designations

## Measurements

In [ ]:
measurements = load_wfs_layer("zwr_public:resultaatsen")
measurements = measurements.drop(columns=['id', 'datum_geplande_monstername', 'monitoring_datum_id', 'monitoring_plan_id', 'monster_id', 'type_object_id', 'geometry'])
measurements

In [ ]:
measurements.dtypes

In [ ]:
measurements = measurements.sort_values(["zwemwaterlocatie_id", "object_begin_tijd", "type_object_code"])
measurements = measurements.drop_duplicates(subset=["zwemwaterlocatie_id", "type_object_code"], keep="first")
measurements

## Preventative measures

In [ ]:
preventative_measures = load_wfs_layer("zwr_public:zwemplek_maatregel")
preventative_measures = preventative_measures.drop(columns=['id', 'bw_code', 'object_type', 'toelichtingstandaardextended', 'type', 'geometry', 'zwemplek_naam'])
preventative_measures

In [ ]:
unique_prevention_codes = preventative_measures.drop_duplicates('toelichtingstandaard')
unique_prevention_codes['toelichtingstandaard']

In [ ]:
messages = preventative_measures.dropna(subset=['toelichtingpubliek'])
messages = messages[['zwemwaterlocatie_id', 'toelichtingpubliek']]
messages

In [ ]:
locs = load_wfs_layer("zwr_public:zwemwaterlocaties")
locs = locs.drop(columns=["id", "datum", "objecttype", "zwl_acht_info"])
locs

## Datamodel